## Import Library

In [ ]:
!pip install -q transformers
!pip install -q simpletransformers
!pip install -q datasets
!pip install sentencepiece
!pip install torch

     |████████████████████████████████| 4.7 MB 7.6 MB/s 
     |████████████████████████████████| 101 kB 12.1 MB/s 
     |████████████████████████████████| 6.6 MB 38.3 MB/s 
     |████████████████████████████████| 596 kB 52.3 MB/s 
     |████████████████████████████████| 249 kB 7.4 MB/s 
     |████████████████████████████████| 1.8 MB 42.3 MB/s 
     |████████████████████████████████| 1.3 MB 39.8 MB/s 
     |████████████████████████████████| 43 kB 1.8 MB/s 
     |████████████████████████████████| 9.1 MB 46.9 MB/s 
     |████████████████████████████████| 365 kB 42.0 MB/s 
     |████████████████████████████████| 181 kB 54.0 MB/s 
     |████████████████████████████████| 157 kB 67.5 MB/s 
     |████████████████████████████████| 63 kB 2.1 MB/s 
     |████████████████████████████████| 157 kB 55.1 MB/s 
     |████████████████████████████████| 157 kB 57.0 MB/s 
     |████████████████████████████████| 157 kB 34.4 MB/s 
     |████████████████████████████████| 157 kB 46.1 MB/s 
     |██████████████

In [ ]:
import pandas as pd
import numpy as np

import tensorflow as tf
tf.get_logger().setLevel('ERROR')

import os
import warnings
warnings.filterwarnings("ignore")

import seaborn as sns
import transformers
from transformers import TFXLNetModel, XLNetTokenizer

from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline
from transformers.tokenization_utils import TruncationStrategy

from sklearn.model_selection import KFold as kfold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve

from simpletransformers.classification import ClassificationModel, MultiLabelClassificationModel
import string
import sentencepiece
import logging
import sklearn

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Load Data

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/data_filter.csv')
pd.set_option('display.max_colwidth', -1)
df = df[['filter', 'label',]]
df.columns = ['text', 'label']
df.head()

,text,label
0,bisa coba daftar canggung uang bilang bukan sekian receh biar kali wisuda pada pergi kosong transkrip penuh semester ini analis ajar mikro biotek pangan beliau sesuai salah dosen favorit kecewa wajar seleksi kecuali protes cerita dana per solusi jadi suka anak hasil dua beda jongkok bela baik ganti kerja syarat kurikulum praktik dunia nyata langsung dapat ilmu mungkin tidak kampus kasih saran takut selesai janji lihat jadwal gatal saudari perempuan jam tubuh praktikum beres bingung sedang saya otak judul hai juga standar orang jiwa montir jujur buka ketik bintang rambut spasi bagus lupa ukur begitu dengki ikat hari astagfirullah sempat besok dasar format bangun tidur dia teknologi jatuh cinta kuliah bidang cuti akademik punya itu swasta aneh kimia teknik usaha sahabat Tuhan bantu kalau tanya awas keluh milik sah mereka sedih turun banget tingkat pacar santai susah seminar energi sistem data tes laut buta warna gratis kangen suasana bareng ke musnah audiensi sudah ribu kuota genap direktur bicara semangat kalimat tahu sengaja ngebut awan doa dengar bumi kamu sih lelucon lusa hati oke fokus ah untuk waktu di ampuh mimpi yang alias kue kabar duka dagang bantal formalitas aren nilai eh grup cewek ganteng tapi rajin seperti toko pasti bahan evaluasi bikin alangkah bahasa lebih aplikasi publik ingat omong sama banyak staf muda bandung barat mantap laboratorium karat logam kain akibat lembab basah badai,1
1,bisa alah perempuan muka buruk jalang kami ini pada lagi umur dapur bau arak teman aneh jam dari Anda lantai dan kemudian saudara film bagaimana jago langsung anjing kontol gang kanker cek faktur lagu kadang hakim orang sadar ampun aplikasi sedih mati itu mitos favorit Tuhan siapa polisi cerita saudari bikin loyang terima jadi ketek awan lupa nawaitu mandi ahli khalifah tolong harap cerah selesai enak sih senang asli hanya bicara rambut kali dada mungkin bukan cukup untuk bahkan punk tetapi memang utama hilang versi hitam komedi psikologis drama kucing tidak sengaja begitu taruh referensi hukum bagus keluarga kaum rumah rusak hitung jarang ayo dengar saya ke sesak pusing baca carut marut bosan tim makan tapi haji cari gambar lihat beda sayang televisi adegan getek bioskop elektronik minum balsem siang hidup data dia atau hasil kuwu sekarang sahabat malam anak normalisasi dasar rakyat beli heran sudah Januari satu kerucut masih hal adalah album redaksi cepat krim tahu saja suka kasihan sumpah peng pita mana jijik deh bawa putra brengsek kupat pagi dulu suci belum bodoh lelaki usia grup nama salah paket kalian akhir hari turun kita manusia makhluk rame hamil setan eh ketang geleng penjara bangsat habis laba-laba pria peran krusial fase asu seri buta sangat demi sesuatu gelap tidur masuk super kenyal kuliah maksimal target pribadi utara Selatan sampai subuh utang poker tinggal kembali kuat tali tambang ketemu yang istirahat skripsi santai sembari hai kantin lontong idiom sirna pandemi,0
2,orang ke bisa coba Januari mereka suster stiker kulkas jaman pergi pondok puasa bukan mandi fokus makan teman hadiah kena azab begitu mahasiswa telekomunikasi gaya ekonomi sulit apakah malaikat susah setrum mengapa tong saudari rel aksen kocok sudah panci masuk atas samping ajak bicara koki bantal guling punya pacar pagi ribut untung sate aduh serang ayo legenda gendong epik utama jam elite banyak haji besok final kopi pusing hal untuk adalah serba pria marah putih sekarat rusuh umur di jarum neurologi doa teriak ikan emas beda kolam keretan nomor tinggi sekolah ada tulus baik syariah jagat raya air habis danau galau cari dosen selagi sepi banget hujan mari ayam nasi ribu siklus skripsi butuh sistem baru tentang sadar ini arek parkir jalan kaki sehat menang bibir ketuk informasi tahu lagi porno akun ingat oke bangga satu kita sekarang juga hidup langsung modal kuota lagu aman dan suara bilang enak mudah tulang lunak krom dengan pulsa yang asrama gudang jendela toilet biar sebelah pada tetapi bambang,1
3,coba yang orang hebat banyak gam

In [ ]:
df['text'] = df['text'].astype(str)

## Start

k-fold splitting

```
acc_score = []

for train_index , test_index in kf.split(X):
    X_train , X_test = X.iloc[train_index,:],X.iloc[test_index,:]
    y_train , y_test = y[train_index] , y[test_index]
     
    model.fit(X_train,y_train)
    pred_values = model.predict(X_test)
     
    acc = accuracy_score(pred_values , y_test)
    acc_score.append(acc)
     
avg_acc_score = sum(acc_score)/k
 
print('accuracy of each fold - {}'.format(acc_score))
print('Avg accuracy : {}'.format(avg_acc_score))
```

[ref k-fold splitting](https://www.askpython.com/python/examples/k-fold-cross-validation)

In [ ]:
X = df

k = 10
kf = kfold(n_splits=k, random_state=None)
 
for train_index , test_index in kf.split(X):
    train_df , test_df = X.take(list(train_index)), X.take(list(test_index))

print(train_df.shape)
print(test_df.shape)

(144, 2)
(15, 2)


In [ ]:
k = 10
kf = kfold(n_splits=k, random_state=None)
 
for train_index , test_index in kf.split(train_df):
    train , val_df = train_df.take(list(train_index)), train_df.take(list(test_index))

print(train_df.shape)
print(val_df.shape)

(144, 2)
(14, 2)


In [ ]:
train_df

,text,label
0,bisa coba daftar canggung uang bilang bukan sekian receh biar kali wisuda pada pergi kosong transkrip penuh semester ini analis ajar mikro biotek pangan beliau sesuai salah dosen favorit kecewa wajar seleksi kecuali protes cerita dana per solusi jadi suka anak hasil dua beda jongkok bela baik ganti kerja syarat kurikulum praktik dunia nyata langsung dapat ilmu mungkin tidak kampus kasih saran takut selesai janji lihat jadwal gatal saudari perempuan jam tubuh praktikum beres bingung sedang saya otak judul hai juga standar orang jiwa montir jujur buka ketik bintang rambut spasi bagus lupa ukur begitu dengki ikat hari astagfirullah sempat besok dasar format bangun tidur dia teknologi jatuh cinta kuliah bidang cuti akademik punya itu swasta aneh kimia teknik usaha sahabat Tuhan bantu kalau tanya awas keluh milik sah mereka sedih turun banget tingkat pacar santai susah seminar energi sistem data tes laut buta warna gratis kangen suasana bareng ke musnah audiensi sudah ribu kuota genap direktur bicara semangat kalimat tahu sengaja ngebut awan doa dengar bumi kamu sih lelucon lusa hati oke fokus ah untuk waktu di ampuh mimpi yang alias kue kabar duka dagang bantal formalitas aren nilai eh grup cewek ganteng tapi rajin seperti toko pasti bahan evaluasi bikin alangkah bahasa lebih aplikasi publik ingat omong sama banyak staf muda bandung barat mantap laboratorium karat logam kain akibat lembab basah badai,1
1,bisa alah perempuan muka buruk jalang kami ini pada lagi umur dapur bau arak teman aneh jam dari Anda lantai dan kemudian saudara film bagaimana jago langsung anjing kontol gang kanker cek faktur lagu kadang hakim orang sadar ampun aplikasi sedih mati itu mitos favorit Tuhan siapa polisi cerita saudari bikin loyang terima jadi ketek awan lupa nawaitu mandi ahli khalifah tolong harap cerah selesai enak sih senang asli hanya bicara rambut kali dada mungkin bukan cukup untuk bahkan punk tetapi memang utama hilang versi hitam komedi psikologis drama kucing tidak sengaja begitu taruh referensi hukum bagus keluarga kaum rumah rusak hitung jarang ayo dengar saya ke sesak pusing baca carut marut bosan tim makan tapi haji cari gambar lihat beda sayang televisi adegan getek bioskop elektronik minum balsem siang hidup data dia atau hasil kuwu sekarang sahabat malam anak normalisasi dasar rakyat beli heran sudah Januari satu kerucut masih hal adalah album redaksi cepat krim tahu saja suka kasihan sumpah peng pita mana jijik deh bawa putra brengsek kupat pagi dulu suci belum bodoh lelaki usia grup nama salah paket kalian akhir hari turun kita manusia makhluk rame hamil setan eh ketang geleng penjara bangsat habis laba-laba pria peran krusial fase asu seri buta sangat demi sesuatu gelap tidur masuk super kenyal kuliah maksimal target pribadi utara Selatan sampai subuh utang poker tinggal kembali kuat tali tambang ketemu yang istirahat skripsi santai sembari hai kantin lontong idiom sirna pandemi,0
2,orang ke bisa coba Januari mereka suster stiker kulkas jaman pergi pondok puasa bukan mandi fokus makan teman hadiah kena azab begitu mahasiswa telekomunikasi gaya ekonomi sulit apakah malaikat susah setrum mengapa tong saudari rel aksen kocok sudah panci masuk atas samping ajak bicara koki bantal guling punya pacar pagi ribut untung sate aduh serang ayo legenda gendong epik utama jam elite banyak haji besok final kopi pusing hal untuk adalah serba pria marah putih sekarat rusuh umur di jarum neurologi doa teriak ikan emas beda kolam keretan nomor tinggi sekolah ada tulus baik syariah jagat raya air habis danau galau cari dosen selagi sepi banget hujan mari ayam nasi ribu siklus skripsi butuh sistem baru tentang sadar ini arek parkir jalan kaki sehat menang bibir ketuk informasi tahu lagi porno akun ingat oke bangga satu kita sekarang juga hidup langsung modal kuota lagu aman dan suara bilang enak mudah tulang lunak krom dengan pulsa yang asrama gudang jendela toilet biar sebelah pada tetapi bambang,1
3,coba yang orang hebat banyak gam

In [ ]:
test_df

,text,label
144,alah bisa dengan dia film itu kamu kelamin lancar mantap orang salah sudah suka,1
145,alam benar film gengsi gung hati hidup jalur kali lombok memori nomor nyaman pakai pandang rezeki sang sih sinyal sudut susah tuan turun udara ulang,1
146,adu bangsat buruk dewa jadi kunjung lawan nasib rencana susu tingkat,0
147,aduh ambulans anak Anda aneh anjing badan bagaimana bahagia baik bandung begitu benar bicara bikin bodoh bos bugar bukan butuh capai cari cukup dapat deh dewasa dia diam dosen ego egois enak fokus gampang gara gila habis ingkar ini intim irit itu jadi jalan jam janji jeli joging kabar kagum kalau kali kalong kamu kangkung kasih kasihan kawan ke kelua keluarga kirim konsentrasi kontol kucing kuliah lampu lihat lucu mahasiswa malam malas mati menit metalurgi muncrat niat orang pacar palsu per perang percaya perempuan polos pulang pusing racun revisi risih sabar sahabat saja salah sampai sangat saya sebentar sedih sekarang selagi selalu selesai semester seperti seram sikap skripsi suka sumpah susah taksir tambun tapi teman tetapi tidak tidur tinggal tua turun uang umur utama vaksin,1
148,aduh akhir aktor alah ampe astaga atau badai bahagia balon batuk bayar begitu benar biarawan bikin bisa budi buruk butuh cabang cepat cinta cowok dan dari dasar dunia fakta gelap gila habis hari hati herba hidup hobi ikat ingin ini itu jadi kami karma kenal kota kotak krim lah laik langsung lari maaf makan memori menang mereka motor nomor obat opa pada parah penuh pergi pusat sabar saja sampai sebagai sendiri seru suami sudah suka takut tangan taring tawa tes tetapi tidak toge total tubuh,1
149,-nya bagaimana bagus bentuk cantik cinta daging dalam dia handal isu itu jadwal jauh kami kawin ke klarifikasi lidah makan momen pagi paket pasti puring saja segar teori,1
150,acara alat amal ambil antek astronomi bang bawa beda beliau belum benar bikin bilang biru bisa buah bukan bumi cepat cerita daerah derajat di erupsi fenomena film ganti garis gede gempa gerhana gung gus halus hanya hari hasil hati heran hisab ilmuwan imam informasi istana jadwal jalan jam juragan juz kabar kadang kader kado kalah kamar kartu kasih kemudian ketupat komentar kotak kriteria kritik lajang langka lidah lompat maju mandi masuk matahari maulid memang menit merah merek metode minimal minum mobil muktamar mundur nabi nama negara non nusantara opor orang organisasi pandemi parah percaya persis pertiwi pesan pindah politik positif pribadi rakat resmi rukyat rupiah salah satu sebelah seismograf sekolah seminar seperti sponsor subuh sudah surat tapi tektonik tidak tukar untuk usaha walah witir yang yuk,1
151,acara aduh ambil Anda aneh anjing anyelir baca badan bagaimana bagus bahagia bahan bangkai bangsat bareng batuk bawa belanja beli beres bicara bingung bisa bocah bodong bukan buset cantik capai cek ceng cepat dan dapat darah datang deh dengar detik dia diare dingin divisi doi dunia eh emosi enak energi festival foto gladi habis hangat hari hati hilang individu ingin istirahat jadi jam Januari jauh jitak jiwa jual kabar kadang kalau kalau-kalau kaldu kali kamu kasih ke keluarga kesal ketel klise kosong kotor kurang laboratorium lagu lihat makan makhluk malaikat malam marah mati memang memori mengerti menit mereka meriang minum motor muncul oke orang organisasi pagi panik penuh pergi pilek potensi proses pulang pusing racun radang rumah sadar salah sampai sangat santai saya sebentar sedih sekarang sekian sembuh seminari sesak sesuatu sial siang sih sore sudah suka sumpah suruh takut tangan tanggung tanya teman terbang terima tidak tidur tolong tugas wasalam yang yuk,1
152,aduh angsa asam awet ayo biaya bisa bukan buruk dan fase gas gratis hari harus jeng keranjang lak langit makan margin miring pagi pedas ragu sahabat sahur selagi seterika siang sih sore taruh ulang utama,1
153,angin anjir asli ayah bandung banget beasiswa benar besok bingung bisa bodoh brutal bunga capai cari cerita dari dengan di eh emak estetis fiksi galau galon gas Gemini gendut gi

In [ ]:
val_df

,text,label
130,air aku anggun atau baik dan dari detik favorit gen hari ini istirahat itu kebun kelas keluarga ketemu kota muda putih renang salah tahun tinggi Tuhan tukang voli yuk,1
131,aduh akan aku Anda anjing atau bagus baik banget bareng bawa benar berat besok bisa cantik cari cewek cinta coba cokelat cus dan datang deh dengan dia diam dusta ganti gede gelap hanya hitam ingin ini itu jam jangan jika juri kalau kami kampung kamu kandang ke kembali kemudian kep komputer kucing lagu langsung lapar lembap lima lucu makan mangan mantan manusiawi marah mari matriks mau maya mengapa mungkin nenek nomor nyala nyata pa pacar paham paku pendek pos putih rambut rumah santai saran saya sedang sekarang sekedar selesai senang seni seram serius skripsi sudah sumpah susah syah tetapi tidak uang umur warna wawancara ya,1
132,adalah Anda asyik ayo badan banyak begitu bisa bunga cek daftar dia dua eror ganti garis gratis hai hal hidup hilang jadi jam kabar kakak kali kangen kawan ke kemudian kontak lamar lumayan makan mengapa minum musik nomor obat oke penuh pin pria rencana selalu sudah suka teman tetapi tidak tidur utama waktu yang,1
133,akan akhlak alah anak Anda api besok bis bisa bodoh bukan cahaya cantik cari cinta cokelat dan dasar efektif efendi fajar fakta fiksi gampang gelap gerung hidup imam indah ini istimewa jatuh juga kasar kereta kosong kritik kuliah labil lajang langka layak libur lidah lihat manis manusia masyarakat media mental mulia ngeri nikah orang pahit peduli perempuan pergi produk putih salah sama selalu selingkuh semangat senja sesuai setia siapa sosial suka sulit tinggal Tuhan utama waktu wanita yang,1
134,acara aib akan aksi alah ampe Anda aplikasi atau bablas bagaimana bahkan bangga bangsat banyak barang bareng bau begitu benar biar bikin bisa butuh cabai cacing candu cari contoh daerah dan dapat dapur dari datang dengan dewasa di dia doi dominan final fobi fulus geli gila hal halaman hangat hanya heboh hewan humoris impresi ingin ini itu jadi janji Januari jarak kabur kadang kaki kalau kali kami kamu kasih ke kecil kenal kenapa kendala kendali kep kepincut kerja kita klik korban kuah kursus lagi langsung layak lihat lucu maklum maksud manis manusia masuk mata-mata mau maya melata mukena muluk munafik niat nihil nyaman oke orang pada panas peduli percaya perempuan perilaku pikir pintar pola progres rakyat rambut rendah risih roda rumit sakit salah sampai sangat satu saya sayang sekarang selalu selera sepele seperti sial siapa singa spesialisasi spons standar suka supaya tahu takdir takut tampan tapi tawon terang tetap tetapi tidak tipe tolong total tulung tunggu turun ubah udang untuk untung utama warna yang,0
135,-nya adalah adem aduh alah alias ambar Anda anti artis atas baca badut bahagia banget banyak beda besok betapa biar bicara bingung bisa bodoh bosan bukan cantik cemas cerah Cina cinta daerah dan dapat dari debut deh dengan dingin diskon doi dom dosen emosional enak energi fokus gila gondrong gravitasi hanya harga hari harimau harus hawar info ingin ini itu jadi jarang jurnal kah kakek kalau kali kambuh kampus kangen kasih kawan kaya ke kena keriting kerja ketemu kirim kompatibel konser konsonan korupsi kota kualitas kuat lagi lagu lambung langit lidah lihat lob lobi-lobi makan makanya makhluk manis masuk memang minimal mobil motor muda mudah musik nusantara nyata oke ongkos orang organisasi pacar pagi pastel penuh perawan perempuan permata persis pilih pita poni poster rantau revisi rilis rumah saja sakit sangat satu saya sayang sayap sebagai sekarang sekitar selamat semenjak senang senyum seperti siap sih sombong sosial suara sudah suka sulit sumpah susah tahta tahu teman tempur tidak tolong tuan Tuhan untuk video waktu warna won xi yang,1
136,ajak anak anjing ayo bangku bangsat bawa biar bisa buang coba dan dengan frekuensi iseng jadi jam kali kapok kasir kawan kemudian kepala kertas kode kosong langsung malu muka muslim orang pacar pesan pilih pusing ramai saya senang seram setia sini sudah teman t

## xlnet

In [ ]:
num_labels = df['label'].nunique()  # number of unique categories in the target column

num_labels, sorted(df['label'].unique())

(2, [0, 1])

In [ ]:
df['label'].value_counts()

1    94
0    65
Name: label, dtype: int64

In [ ]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import logging

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [ ]:
import torch
torch.cuda.is_available()

False

In [ ]:
model_type = 'xlnet'
model_name = 'xlnet-base-cased'

train_args={
    'reprocess_input_data': True,
    "weight_decay": 0.1,
    "adam_epsilon": 1e-8,
    "learning_rate": 5e-5,
    "warmup_ratio" : 0.06,
    "warmup_steps" : 50,
    "scheduler" : "cosine_schedule_with_warmup",
    "overwrite_output_dir": True,
    "use_cached_eval_features": True,
    "train_batch_size": 56,
    "num_train_epochs": 15,
    'evaluate_during_training': True,
    "evaluate_during_training_verbose" : True,
    "eval_batch_size": 56,
    'logging_steps': 5,
    'stride': 0.8,
    'max_seq_length': 128,
    "gradient_accumulation_steps": 2
}

In [ ]:
%%time

model = ClassificationModel(model_type, model_name, args=train_args,num_labels=num_labels, use_cuda=False,)

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['sequence_summary.summary.bias', 'sequence_summary.summary.weight', 'logits_proj.weight', 'logits_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

CPU times: user 1.28 s, sys: 431 ms, total: 1.71 s
Wall time: 5.57 s


In [ ]:
## Train the model

model.train_model(train_df, eval_df=val_df)

  0%|          | 0/144 [00:00<?, ?it/s]

Epoch:   0%|          | 0/15 [00:00<?, ?it/s]

Running Epoch 0 of 15:   0%|          | 0/3 [00:00<?, ?it/s]

Running Epoch 1 of 15:   0%|          | 0/3 [00:00<?, ?it/s]

Running Epoch 2 of 15:   0%|          | 0/3 [00:00<?, ?it/s]

...

Running Epoch 11 of 15:   0%|          | 0/3 [00:00<?, ?it/s]

Running Epoch 12 of 15:   0%|          | 0/3 [00:00<?, ?it/s]

Running Epoch 13 of 15:   0%|          | 0/3 [00:00<?, ?it/s]

Running Epoch 14 of 15:   0%|          | 0/3 [00:00<?, ?it/s]

(15,
 defaultdict(list,
             {'global_step': [1,
               2,
               3,
               4,
               5,
               6,
               7,
               8,
               9,
               10,
               11,
               12,
               13,
               14,
               15],
              'train_loss': [0.1241646558046341,
               0.06942970305681229,
               0.12548403441905975,
               0.37150493264198303,
               0.3124661445617676,
               0.13092932105064392,
               0.12418267875909805,
               0.11950940638780594,
               0.10603410005569458,
               0.07275354862213135,
               0.053259581327438354,
               0.05429640784859657,
               0.14308549463748932,
               0.044538673013448715,
               0.11632619053125381],
              'mcc': [0.5773502691896257,
               0.42857142857142855,
               0.14907119849998599,
               

## Eval

In [ ]:
# Evaluate the model
result_xlnet, model_outputs, wrong_predictions = model.eval_model(test_df, 
                                                            acc = sklearn.metrics.accuracy_score,
                                                            )

Running Evaluation:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
# Displaying the model metrics
result_xlnet

{'mcc': 0.14907119849998599,
 'tp': 3,
 'tn': 5,
 'fp': 2,
 'fn': 4,
 'auroc': 0.7142857142857143,
 'auprc': 0.7115646258503401,
 'acc': 0.5714285714285714,
 'eval_loss': 0.9225683212280273}

In [ ]:
# Creating a dataframe that stores the model details
model_performance = pd.DataFrame(columns=['Model', 'Test Accuracy', 'Test Loss','MCC', 'AUROC', 'AUPRC', 'Precision', 'Recall', 'F1'])

In [ ]:
def acc(a, b, c, d):
  acc = (a+b) / (a+b+c+d)
  return acc

def prerec(a, b):
  hasil = a / (a+b)
  return hasil

def f1score(a,b):
  f1 = 2*(a*b)/(a+b)
  return f1

In [ ]:
tp = result_xlnet['tp']
tn = result_xlnet['tn']
fp = result_xlnet['fp']
fn = result_xlnet['fn']

model_performance = model_performance.append({'Model':'XLNet with Data Augmentation',
                                              'Test Accuracy': result_xlnet['acc'],
                                              'Test Loss': result_xlnet['eval_loss'],
                                              'MCC': result_xlnet['mcc'],
                                              'AUROC': result_xlnet['auroc'],
                                              'AUPRC': result_xlnet['auprc'],
                                              'Precision': prerec(tp, fp),
                                              'Recall': prerec(tp, fn),
                                              'F1' : f1score(prerec(tp, fp), prerec(tp, fn))                                  
                                              }, ignore_index=True)

model_performance

,Model,Test Accuracy,Test Loss,MCC,AUROC,AUPRC,Precision,Recall,F1
0,XLNet with Data Augmentation,0.5,0.673566,0.0,0.673469,0.630875,0.5,0.428571,0.461538


## Prediksi

In [ ]:
def persen_depresi(akun):
  jlh = sum(akun)
  ttl = len(akun)
  persen = (jlh / ttl) * 100
  return print('Persentase Akun = ', round(persen, 2), '% \n')

In [ ]:
# Displaying a sample prediction

sample_idx = 0

pred = (test_df['text'].iloc[sample_idx])

# Make predictions with the model
predictions0, raw_outputs = model.predict(pred)

print(f'Sampled Text: {test_df["text"].iloc[sample_idx]}')
print(f'True Class: {test_df["label"].iloc[sample_idx]}')
print(f'Predicted Class : {predictions0}')
print(persen_depresi(predictions0))

  0%|          | 0/1 [00:00<?, ?it/s]

Sampled Text: alah ana film ha iya jalan kamu kelamin lancar mantap orang rekomendasi salah salin sudah suka
True Class: 1
Predicted Class : [1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 0 1 0 0 1 0 0 1 0 0 1 0 0 1 0 0 1 0 0 1 0 0
 1 0 0 1 0 0 1 0 0 1 0 0 1 0 0 1 0 0 1 0 0 1 0 0 1 0 0 1 0 0 1 0 0 1 1 0 1
 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Persentase Akun =  37.23 % 

None


In [ ]:
len(predictions0)

184

In [ ]:
# Displaying a sample prediction

sample_idx = 1

pred = (test_df['text'].iloc[sample_idx])

# Make predictions with the model
predictions1, raw_outputs = model.predict(pred)

print(f'Sampled Text: {test_df["text"].iloc[sample_idx]}')
print(f'True Class: {test_df["label"].iloc[sample_idx]}')
print(f'Predicted Class : {predictions1}')
print(persen_depresi(predictions1))

  0%|          | 0/1 [00:00<?, ?it/s]

Sampled Text: segs asui emng nyaman sih gengsi harus dijaga sedang bersingah hati kembali pada sang tuan skip ada halu pakai semvak ha kok akujuga no filter neded kak gung merasakan kali perjalan jalur udara sangat berkesan perjalan tidak ada berhenti melihat keindahan alam indonesia sudut pandang berbeda semoga rezeki mengulang memori ini journey keknya related sih di film susah sinyal mau nenangin larinya lombok emang sih temenku ulang ucapin iya
True Class: 1
Predicted Class : [1 1 1 1 0 1 0 0 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

In [ ]:
len(predictions1)

438

In [ ]:
# Displaying a sample prediction

sample_idx = 2

pred = (test_df['text'].iloc[sample_idx])

# Make predictions with the model
predictions2, raw_outputs = model.predict(pred)

print(f'Sampled Text: {test_df["text"].iloc[sample_idx]}')
print(f'True Class: {test_df["label"].iloc[sample_idx]}')
print(f'Predicted Class : {predictions2}')
print(persen_depresi(predictions2))

  0%|          | 0/1 [00:00<?, ?it/s]

Sampled Text: dipikir rencana madara terlalu buruk bangsat wk lawan adu nasib terlalu jika terikat anjim horor tingkat dewa pdkt tidak kunjung jadian olah kaya yang menderita dunia
True Class: 0
Predicted Class : [1 1 1 1 0 1 0 0 1 1 1 1 1 1 1 0 1 0 0 0 0 0 0 0 0 0 1 0 1 0 0 1 0 1 0 0 1
 0 1 0 1 0 0 0 1 0 0 1 1 0 1 0 0 0 0 1 0 1 0 0 0 0 0 1 0 0 0 1 0 1 1 0 1 0
 1 1 1 1 1 0 1 0 0 0 1 1 0 1 1 0 0 0 1 0 1 1 1 0 1 1 1 1 0 1 0 0 1 0 0 0 0
 1 0 1 0 1 0 0 1 0 0 0 1 0 0 0 1 0 1 0 0 0 0 1 0 0 0 0 1 0 0 1 1 1 0 0 0 0
 1 0 0 0 0 0 0 0 1 0 0 1 0 1 0 0 1 0]
Persentase Akun =  40.96 % 

None


In [ ]:
len(predictions2)

166

In [ ]:
# Displaying a sample prediction

sample_idx = 3

pred = (test_df['text'].iloc[sample_idx])

# Make predictions with the model
predictions3, raw_outputs = model.predict(pred)

print(f'Sampled Text: {test_df["text"].iloc[sample_idx]}')
print(f'True Class: {test_df["label"].iloc[sample_idx]}')
print(f'Predicted Class : {predictions3}')
print(persen_depresi(predictions3))

  0%|          | 0/1 [00:00<?, ?it/s]

Sampled Text: jels termasuk bimbingan revisi diturutin dihargain ngelakuin irit gila kali mod skirpsian it okay but halo penjual netflix halo netflix nay valo jnm njkl jm hjk fghjki dftyjtydjt drthdtrh dnfrty cvbncv jgmfjf xcvb awefg aewsfasef sgeswe rthdfg tryurtyu cvb ghjklk hgjk fgh dfg erg eqrtwert ertwer werywerty ertywy drfthdrtj dfghrseg benar pusing down pacar ahha dari tipe kaya kak mengajak main teman amp ngechat duluan nanyain kabar lama fedbacknya chat duluan gakan ada percakapan diam aja gangajak bukber main amp kabar percaya elu kelakuan ingkar janji sahabat dewasa punya kesibukan msing orang stay keadan apapun perempuan kontol egois sedih kasih merasakan dikasih disayang bismilah sempro malam berantem bisa kasih terbaik untuk kedepanya responya meningikan diri sendiri nada bicara membentak harus peduli its your mistake but you always blame nato fck your ego pacar mau he penganguran bodoh mulai tahun ada deket teman smp teman kuliah fake butuh kalau saya ada duit bodoh ka

In [ ]:
len(predictions3)

2588

In [ ]:
# Displaying a sample prediction

sample_idx = 4

pred = (test_df['text'].iloc[sample_idx])

# Make predictions with the model
predictions4, raw_outputs = model.predict(pred)

print(f'Sampled Text: {test_df["text"].iloc[sample_idx]}')
print(f'True Class: {test_df["label"].iloc[sample_idx]}')
print(f'Predicted Class : {predictions4}')
print(persen_depresi(predictions4))

  0%|          | 0/1 [00:00<?, ?it/s]

Sampled Text: fagh tikhtokh ai laik nurul budi aluka cowok anjim oemje wakwau pengen menantea pen lari hujanya bikin mes nugas si aduh suka tertawa ketebak pengen staycation so bad gila si ampe ditup knpa hati cenat cenut memikirkan tes huft ahhaha si dagdigdug hatiq karma oh karma kwangso knp si drakor dark semua dark yak wn didepan yuhu soswet ugha jk pls cepet bat alah udh end opa armin yg dibawa toge obat batuk aowk tawa puseng world championship diadain excited takot ih semoga mas sepu hepi ending gilbert hidup oemji utu gemesh klean ga like begitu mas bri aduh suami badai pisan cinta dah gapaham bisa nya yami diculik emngnya menonton uwu oho bucin mulu la yg la selengkih ga sabar tngl aowk hapy hobi day hbd norman hidup astaga apa si solo leveling nti dibikin anime omeje cie karma closer overthinking narak bayar win saja halu astaga dah gila nti ye when apa si yuzu dating ‍ tahu si dah berteman mantap yrt ot eb reteb alnr woeay luv hapytaehyungday hbd iya butuh holiday hapy brigh

In [ ]:
len(predictions4)

1531

In [ ]:
# Displaying a sample prediction

sample_idx = 5

pred = (test_df['text'].iloc[sample_idx])

# Make predictions with the model
predictions5, raw_outputs = model.predict(pred)

print(f'Sampled Text: {test_df["text"].iloc[sample_idx]}')
print(f'True Class: {test_df["label"].iloc[sample_idx]}')
print(f'Predicted Class : {predictions5}')
print(persen_depresi(predictions5))

  0%|          | 0/1 [00:00<?, ?it/s]

Sampled Text: kobospace puring ingin kawin kobospace eskrimnya dirakit pengalihan isu bagus kobo mana jadwal makan seharian tungu twt kobo makan kobo slebew kanaeru shesh bisanya teori anata takae kobo amp chan shap yopi mangats kobo shesh hachama fresh meat earphone im die thankyou forever yopi pengantar paket handal mau menemani paket kuambil antar he go back want to be french fries nani klarifikasi moment yes we can pasti obisa kita cinta iofi he ikz obisa pagi yopi its how to use anya properly xd
True Class: 1
Predicted Class : [1 1 1 1 0 1 0 0 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0

In [ ]:
len(predictions5)

491

In [ ]:
# Displaying a sample prediction

sample_idx = 6

pred = (test_df['text'].iloc[sample_idx])

# Make predictions with the model
predictions6, raw_outputs = model.predict(pred)

print(f'Sampled Text: {test_df["text"].iloc[sample_idx]}')
print(f'True Class: {test_df["label"].iloc[sample_idx]}')
print(f'Predicted Class : {predictions6}')
print(persen_depresi(predictions6))

  0%|          | 0/1 [00:00<?, ?it/s]

Sampled Text: pantesan akun kalo ikuti seminar pra muktamar amin tahu pesan beliau muhamadiyah harus independen mendukung jangan masuk istana formula sukses eh banyak akui kasih sponsor wk cerita basi komentar gini masuk kamar mandi lompat auto beda alam bandung bang jawa masuk sunda empire wk ilmuwan nentuin gerhana muhamadiyah pakai hisab giliran pas gerhana percaya giliran ginian percaya bilang mah tungu rukyat hisab tentu gerhana jadwal sholat pakai hisab disuguhin minum iya gp disugihin buah kadang dimakan fyi ilmuwan astronomi memakai hisab mentukan terjadinya fenomena astronomi gerhana matahari lah emang hasil hisab sudah derajat menuhi kriteria minimal derajat pemerintah hisab tepat bakal dipakai nentuin terjadi gerhana masjid² gede metode begitu ngelaksanain rakat ganti imam ambil rakat sudah jam dibaca surat one day one juz jd mau mundur pas gak bawa² ormas kader muhamadiyah masuk politik satupun kader muhamadiyah politik bawa² muhammadiyah iya muhamadiyah organisasi independ

In [ ]:
len(predictions6)

2883

In [ ]:
# Displaying a sample prediction

sample_idx = 7

pred = (test_df['text'].iloc[sample_idx])

# Make predictions with the model
predictions7, raw_outputs = model.predict(pred)

print(f'Sampled Text: {test_df["text"].iloc[sample_idx]}')
print(f'True Class: {test_df["label"].iloc[sample_idx]}')
print(f'Predicted Class : {predictions7}')
print(persen_depresi(predictions7))

  0%|          | 0/1 [00:00<?, ?it/s]

Sampled Text: iya anjir jitak orang wk nyesel baca ginian kesal anj bocah prik anjer anjir tidur nangung bgt ya badanya gemeteran merenung kemarin pergi kemana ya aing beli ya jir melow iya aing udh deg²an email lab pas cek foto mikir proses ngirim direload terus²an fotonya sampe kanya kasih klise ketemuan luar ngejelasin ngesek bgt racun hanasui menyangupi permintan takut keteran potensi bagus gatau tuh aprove enggapun ya deh tidakapa bangke iya sih dingetin sulit wk iya si angel nanya tepar vera enga eh beres acara penyuluhan sore rumah rebahan tiba kebangun jam malem ikutan gladi kotor seminar iya katanya tepar ra afk bir kedut² iya anjir takut ya gitu rasanya kalo ngeracun gua menabung kurangin jan² wk berpikir sekian menit wkk meruntuhkan pertahan nonton now playing festival wk dibikin kesel gak mengerti tolong suruh nonton di day apasih anj nyambung kesal ada deh wkkw bahan julid anjr ha gak bisa jiwa ingintahu meronta ha pas song sama menyebalkan sebel sekarang dengar lagu kaya 

In [ ]:
len(predictions7)

3084

In [ ]:
# Displaying a sample prediction

sample_idx = 8

pred = (test_df['text'].iloc[sample_idx])

# Make predictions with the model
predictions8, raw_outputs = model.predict(pred)

print(f'Sampled Text: {test_df["text"].iloc[sample_idx]}')
print(f'True Class: {test_df["label"].iloc[sample_idx]}')
print(f'Predicted Class : {predictions8}')
print(persen_depresi(predictions8))

  0%|          | 0/1 [00:00<?, ?it/s]

Sampled Text: ujanya awet masama bana tertawa pankapan main lagi ayo nab pedes si ha iya ba pagi siang selagi masi shift sore ayo ba ragu kak kerja seterika menikmati hari wfh ha moso claudya amp kiwil wes saya gok kos wae italic adeh ayangbeb wk yokla main lin duh margini jeng kos ha meira maima maira wk kapan laq wk teko pedot ndak ra wingi pedot swalow ba gitu kemarin ngajakin main wk asem marai ra kolu makan sahur kejedug knpa bestie ayo keluar oink jogja lak gas pewe fre trial tidak jogja put gas pibesdey re lali rid ha cofetography lg fase ini sih yaw hope you ve made peace in your own way
True Class: 1
Predicted Class : [1 1 1 1 0 1 0 0 1 1 1 1 1 1 1 0 0 0 1 0 0 0 0 0 1 0 0 1 1 0 0 1 0 0 0 1 0
 0 0 0 1 0 0 1 1 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 1 0 1 0 0 1 1 0 1 1 1 0
 1 0 1 1 1 1 1 1 1 1 1 1 0 1 1 0 0 1 0 1 1 1 0 1 1 0 1 1 0 1 0 1 0 1 1 0 1
 0 0 1 1 0 1 1 0 0 0 0 0 1 0 1 0 0 1 0 0 0 0 0 1 0 0 0 1 0 0 1 0 1 1 0 0 1
 0 1 0 0 0 0 0 1 1 0 1 0 0 0 1 0 1 0 0 0 0 0 1 0 0 1 0 0 0 0 0 1 

In [ ]:
len(predictions8)

588

In [ ]:
# Displaying a sample prediction

sample_idx = 9

pred = (test_df['text'].iloc[sample_idx])

# Make predictions with the model
predictions9, raw_outputs = model.predict(pred)

print(f'Sampled Text: {test_df["text"].iloc[sample_idx]}')
print(f'True Class: {test_df["label"].iloc[sample_idx]}')
print(f'Predicted Class : {predictions9}')
print(persen_depresi(predictions9))

  0%|          | 0/1 [00:00<?, ?it/s]

Sampled Text: seperti seul bisa meramalkan ku ahi avoskin gasie ha kebiasan bereskpetasi tingi woi sekali syudah nonton dik emak bedanya emak keropi ndak berbau warna hijau hiks bm jco itu sapinya masuk angin ga wow bodoh kebulatan kepala aneh weh terbahak brutal anying panik tuh wifi keputus kucing kontrakan banyak iya si gendut manies poni besok siapa penghuni baru capai sangat disuruh remed story orang shit hadeh capai haluin fiksi seperti pramadi dimana amin hapybirthday gemini boy nya dikenal sebagai proposal ha sial lagi sial jompo gue gils tengorokan lagi pikir hebat tru min act of servis ihiw waduh kamar juga kalerpul sprei starwars selimut keropi teman sprei pink bunga belum lagi lampu tmblrnya asik masuk estetik ye ha temen bertengkar cowonye ngapa sih yang disuruh ngebujuk tidak disuruh cari info waduh lama tidak diginin biasanye tidak amang warung kopi ye amang galon gini tungu diluar step bri ngapa banyak paragon dah yalah otak hamba cair anying typonya askakska asli rumah

In [ ]:
len(predictions9)

1449